#  Importing Dependencies

In [1]:
from pytube import YouTube
from pytube import Playlist
from youtube_transcript_api import YouTubeTranscriptApi as yta
import os
from moviepy.editor import *
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import ffmpeg
import glob
import whisper
model=whisper.load_model("base")

In [2]:
cwdpath=os.getcwd()
#print(cwdpath)
Data_path=os.path.join(cwdpath,"ISLTRC_DATA")
temp_path=os.path.join(Data_path,"temp")
tempvid_path=os.path.join(Data_path,"video")
print(temp_path)

E:\SIGN_LANGUAGE_CORPORA_GENERATION\ISLTRC_DATA\temp


# Creating Useful Functions

In [3]:
def get_audio_webm(video):
    audio_streams = video.streams.filter(only_audio=True,file_extension="webm").order_by("abr").desc()
    
    audio_streams = video.streams.filter(only_audio=True, file_extension="webm").order_by("abr").desc()
    for t in audio_streams:
        if(t.abr=="160kbps"):
            audio=t
            break
    return audio

In [4]:
def get_720p_video(yt):
    '''given an object of the class Youtube, return the video with 720p in mp3 format'''

    video_streams = yt.streams.filter(only_video=True, file_extension="mp4", progressive=False).order_by('resolution').desc()
    for t in video_streams:
        if(t.resolution == "720p"):
            video = t
            break

    return video

In [5]:
def download_audio(audio,temp_path):
    audio.download(temp_path)

In [6]:
def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)

In [7]:
def give_time_stamps_subclips(nonsilent_data):
    

    # nonsilent_Data is a list of entries of type [start,end] which denote the non silence 
    ans=[]
    for item in nonsilent_data:
        start=item[0]
        end=item[1]
        start=round(start/1000,2)
        end=round(end/1000,2)
        start=round(start-0.1,2)
        end=round(end+0.1,2)
        temp=[start,end]
        ans.append(temp)
    return ans

In [8]:
def clip(folder_path, video_path, audio_path, time_stamps, remove_orignal=False):
    '''
    given the transcript makes subclips of the audio ,video files and 
    creates their transcript using whisper api
    which are stored at video_path and audio_path
    then clubs them together in seperate folders in the folder_path
    '''

    
    input_video = VideoFileClip(video_path)
    input_audio = AudioFileClip(audio_path)

    for j, (start, stop) in enumerate(time_stamps):
        sub_video = input_video.subclip(start, stop)
        sub_audio = input_audio.subclip(start, stop)

        sub_folder_path = os.path.join(folder_path, "sub_" + str(j))
        os.makedirs(sub_folder_path)
        sub_video_path = os.path.join(sub_folder_path, "video.mp4")
        sub_audio_path = os.path.join(sub_folder_path, "audio.mp3")
        sub_text_path=os.path.join(sub_folder_path,"transcript.txt")

        sub_video.write_videofile(sub_video_path)
        sub_audio.write_audiofile(sub_audio_path)
        result=model.transcribe(sub_audio_path,fp16=False)
        transcript=result['text']
        with open(sub_text_path,"w") as text_file:
            text_file.write(transcript)

    if remove_orignal:
        os.remove(video_path)
        os.remove(audio_path)

# Selecting Youtube playlist to scrape

In [9]:
playlist_url="https://www.youtube.com/playlist?list=PLFjydPMg4Daoarxj631EE2_p_3Wy6I19B"
playlist = Playlist(playlist_url)
#print(list(enumerate(playlist)))

In [10]:
unable_to_download=[]
unable_to_convert=[]
unable_to_get_stamps=[]
unable_to_clip=[]

In [11]:
os.listdir(temp_path)

['1.mp3']

In [12]:
if(len(os.listdir(temp_path))!=0):
            for filename in os.listdir(temp_path):
                file_path = os.path.join(temp_path, filename)
                try:
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.unlink(file_path)
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                except Exception as e:
                    print('Failed to delete %s. Reason: %s' % (file_path, e))
if(len(os.listdir(tempvid_path))!=0):
            for filename in os.listdir(temp_path):
                file_path = os.path.join(temp_path, filename)
                try:
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.unlink(file_path)
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                except Exception as e:
                    print('Failed to delete %s. Reason: %s' % (file_path, e))
                    


In [13]:
os.listdir(temp_path)

[]

In [ ]:
print_progress==False
for i,link in enumerate(playlist):
    i+=1
    #link=link[1]
    
    '''Download audio from video in .webm format'''
    try:                                                                 
        yt=YouTube(link)
        video=get_720p_video(yt)
        video.download(tempvid_path)
        audio=get_audio_webm(yt)
        download_audio(audio,temp_path)
        if print_progress==True:
            print("Download Succesful for video "+str(i)+" \nDownload Succesful for audio "+str(i))
        file_name=os.listdir()[0]
        file_name_org=file_name
        file_path=os.path.join(temp_path,file_name)
        dest_path=os.path.join(temp_path,str(i)+".webm")
    except Exception as e:
        print("Failed to download %s. Reason: %s"%(file_name,e))
        unable_to_download.append((i,file_name))
        continue
        
    '''Rename the audio file''' 
    file_path=os.path.join(temp_path,os.listdir(temp_path)[0])
    dest_path=0s.path.join(temp_path,str(i)+".webm")
    
    os.rename(file_path,dest_path)                                        
    file_path=dest_path
    dest_path=os.path.join(temp_path,str(i)+".mp3")
    if print_progress==True:
        print("Renaming Succesful")
    
    
    '''Convert .webm file to .mp3 file'''
    
    try:                                                                  
        command="ffmpeg -i "+file_path+" "+dest_path
        os.system(command)
        if print_progress==True:
            print("Audio file conversion Succesful")
        
    except Exception as e:
        print("Failed to convert to mp3 %s. Reason %s"%(file_name_org,e))
        unable_to_convert.append((i,file_name_org))
        continue
    
    
    '''Detect Non_silent data in the audio'''
    
    try:           
        
        audio_segment = AudioSegment.from_mp3(dest_path)
 
        normalized_sound = match_target_amplitude(audio_segment, -20.0)
        
        nonsilent_data = detect_nonsilent(normalized_sound, min_silence_len=2000, silence_thresh=-20,seek_step=1)
        if print_progress==True:
            print("Audio file analysis succesful")
        
    except Exception as e:
        print("Failed to get stamps %s. Reason %s"%(file_name_org,e))
        unable_to_get_stamps.append((i,file_name_org))
        continue
    
    '''Delete the .webm file from temp directory''' 
    
    pattern=os.path.join(temp_path,"*.webm")                             
    file=glob.glob(pattern)
    os.remove(file[0])
    
    
    '''Get the timestamps for segmentation'''
    
    timestamps=give_time_stamps_subclips(nonsilent_data)
    
    '''Creating Subclip folders'''
    
    try:
        folder_path=os.path.join(Data_path,"Subclips")
        video_path=os.path.join(tempvid_path,os.listdir(tempvid_path)[0])
        audio_path=os.path.join(temp_path,os.listdir(temp_path)[0])
        clip(folder_path,video_path,audio_path,timestamps,True)
        if print_progress==True:
            print("Clippings Succesful")
    except Exception as e:
        unable_to_clip.append((i,file_name_org))
        print("Failed to clip video %s, Reason %s"%(file_name_org,e))
        continue
        
    
    print("\n\n")
    print("-"*50)
    print("\n\n")
           